In [1]:
import pandas as pd

In [2]:
recenzie = pd.read_csv('/content/drive/MyDrive/Škola/MAWO/cvicenie_9/sentiment_fiona.csv', sep=";", index_col=None)

In [3]:
recenzie

,Hodnotitel,Text,Sentiment
0,Doris,Akože outfity má otrasné. Makeup je ale super ...,pozitivna
1,Popoluška,Fiona je super princezná. Je vždy dobrá a milá...,pozitivna
2,Dobrá víla,Fiona je náladová. Toho super princa si nezasl...,negativna
3,Snehulienka,Je super.,pozitivna
4,Pinocchio,"Škoda reči, aj tak tento príspevok cenzúra vym...",negativna
5,Mabel,To čo má za muža. Takúto hroznú ozrutu doniesl...,negativna
6,Kyklop,Outfity čo nosí sú otrasné.,negativna
7,Merlin,Naša dobrá princezná je zároveň aj pekná.,pozitivna
8,Artuš,"Je dobrá, môže byť.",pozitivna
9,Jack,Naposledy mala otrasný účes. Tá farba vlasov j...,negativna


In [4]:
recenzie['Text'][9]

'Naposledy mala otrasný účes. Tá farba vlasov je tiež hrozná.'

In [5]:
!pip install stanza
import stanza
stanza.download('sk')

2021-04-26 12:30:17 INFO: Downloading default packages for language: sk (Slovak)...
2021-04-26 12:30:17 INFO: File exists: /root/stanza_resources/sk/default.zip.
2021-04-26 12:30:19 INFO: Finished downloading models and saved to /root/stanza_resources.


In [6]:
nlp = stanza.Pipeline(lang='sk')

2021-04-26 12:30:19 INFO: Loading these models for language: sk (Slovak):
| Processor | Package |
-----------------------
| tokenize  | snk     |
| pos       | snk     |
| lemma     | snk     |
| depparse  | snk     |

2021-04-26 12:30:19 INFO: Use device: cpu
2021-04-26 12:30:19 INFO: Loading: tokenize
2021-04-26 12:30:19 INFO: Loading: pos
2021-04-26 12:30:20 INFO: Loading: lemma
2021-04-26 12:30:20 INFO: Loading: depparse
2021-04-26 12:30:20 INFO: Done loading processors!


In [7]:
def daj_lemy(vstupny_text):
  vystup = ""
  dokument = nlp(vstupny_text)
  for veta in dokument.sentences:
    for slovo in veta.words:
      vystup = vystup + " " + slovo.lemma
  return vystup

In [8]:
daj_lemy('Spadla hruška zelená, rozbila si kolená')

' spadnúť hruška zelený , rozbiť si koleno'

In [9]:
pozitivne = ''
negativne = ''

In [10]:
for index, row in recenzie.iterrows():
  lemmy = daj_lemy(row['Text'])
  if (row['Sentiment'] == 'pozitivna'):
    pozitivne = pozitivne + ' ' + lemmy
  else:
    negativne = negativne + ' ' + lemmy

In [11]:
pozitivne

'  akýže outfita mať otrasný . makeup byť ale super a vlas vždy upravený .  fion byť super princezná . byť vždy dobrý a milý . fion vždy rád pomôcť .  byť super .  náš dobrý princezná byť zároveň aj pekný .  byť dobrý , môcť byť .'

In [12]:
negativne

'  fion byť náladový . to super princ si zaslúžiť .  škoda reč , aj tak tento príspevok cenzúr vymažať .  to čo mať za muž . takýto hrozný ozrut doniesť do kráľovstvo .  outfita čo nosiť byť otrasný .  naposledy mať otrasný účes . tá farba vlas byť tiež hrozný .'

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
sent_vectorizer = CountVectorizer()

In [14]:
korpus = sent_vectorizer.fit_transform([pozitivne, negativne])

In [15]:
hodnotene_slova = pd.DataFrame(korpus.A, columns = sent_vectorizer.get_feature_names())

In [16]:
hodnotene_slova

,aj,akýže,ale,byť,cenzúr,do,dobrý,doniesť,farba,fion,hrozný,kráľovstvo,makeup,mať,milý,muž,môcť,naposledy,nosiť,náladový,náš,otrasný,outfita,ozrut,pekný,pomôcť,princ,princezná,príspevok,reč,rád,si,super,tak,takýto,tento,tiež,to,tá,upravený,vlas,vymažať,vždy,za,zaslúžiť,zároveň,účes,čo,škoda
0,1,1,1,7,0,0,3,0,0,2,0,0,1,1,1,0,1,0,0,0,1,1,1,0,1,1,0,2,0,0,1,0,3,0,0,0,0,0,0,1,1,0,3,0,0,1,0,0,0
1,1,0,0,3,1,1,0,1,1,1,2,1,0,2,0,1,0,1,1,1,0,2,1,1,0,0,1,0,1,1,0,1,1,1,1,1,1,2,1,0,1,1,0,1,1,0,1,2,1


In [17]:
hodnotene_slova.iloc[1] = (hodnotene_slova.iloc[1] / (-1))

In [18]:
hodnotene_slova.loc['priemer'] = (hodnotene_slova.iloc[0] + hodnotene_slova.iloc[1]) / 2

In [19]:
hodnotene_slova

,aj,akýže,ale,byť,cenzúr,do,dobrý,doniesť,farba,fion,hrozný,kráľovstvo,makeup,mať,milý,muž,môcť,naposledy,nosiť,náladový,náš,otrasný,outfita,ozrut,pekný,pomôcť,princ,princezná,príspevok,reč,rád,si,super,tak,takýto,tento,tiež,to,tá,upravený,vlas,vymažať,vždy,za,zaslúžiť,zároveň,účes,čo,škoda
0,1.0,1.0,1.0,7.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-1.0,-0.0,-0.0,-3.0,-1.0,-1.0,-0.0,-1.0,-1.0,-1.0,-2.0,-1.0,-0.0,-2.0,-0.0,-1.0,-0.0,-1.0,-1.0,-1.0,-0.0,-2.0,-1.0,-1.0,-0.0,-0.0,-1.0,-0.0,-1.0,-1.0,-0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-1.0,-0.0,-1.0,-1.0,-0.0,-1.0,-1.0,-0.0,-1.0,-2.0,-1.0
priemer,0.0,0.5,0.5,2.0,-0.5,-0.5,1.5,-0.5,-0.5,0.5,-1.0,-0.5,0.5,-0.5,0.5,-0.5,0.5,-0.5,-0.5,-0.5,0.5,-0.5,0.0,-0.5,0.5,0.5,-0.5,1.0,-0.5,-0.5,0.5,-0.5,1.0,-0.5,-0.5,-0.5,-0.5,-1.0,-0.5,0.5,0.0,-0.5,1.5,-0.5,-0.5,0.5,-0.5,-1.0,-0.5


In [20]:
veta = "Každá cenzúra hrozná."

In [21]:
veta_lemmy = daj_lemy(veta)

In [22]:
veta_lemmy

' každý cenzúr hrozný .'

In [23]:
veta_token = veta_lemmy.split(' ')

In [24]:
veta_token

['', 'každý', 'cenzúr', 'hrozný', '.']

In [25]:
hodnotene_slova['cenzúr']['priemer']

-0.5

In [26]:
hodnotene_slova['byť']['priemer']

2.0

In [27]:
hodnotene_slova['vždy']['priemer']

1.5

In [28]:
hodnotene_slova['hrozný']['priemer']

-1.0

In [29]:
sentiment = 0;
pocet = 0;
for slovo in veta_token:
  if (slovo in sent_vectorizer.get_feature_names()):
    sentiment += hodnotene_slova[slovo]['priemer']
    pocet += 1
print('Výsledný sentiment = ' + str(sentiment/pocet))

Výsledný sentiment = -0.75


In [30]:
pozitivne

'  akýže outfita mať otrasný . makeup byť ale super a vlas vždy upravený .  fion byť super princezná . byť vždy dobrý a milý . fion vždy rád pomôcť .  byť super .  náš dobrý princezná byť zároveň aj pekný .  byť dobrý , môcť byť .'

In [31]:
negativne

'  fion byť náladový . to super princ si zaslúžiť .  škoda reč , aj tak tento príspevok cenzúr vymažať .  to čo mať za muž . takýto hrozný ozrut doniesť do kráľovstvo .  outfita čo nosiť byť otrasný .  naposledy mať otrasný účes . tá farba vlas byť tiež hrozný .'

In [32]:
recenzie

,Hodnotitel,Text,Sentiment
0,Doris,Akože outfity má otrasné. Makeup je ale super ...,pozitivna
1,Popoluška,Fiona je super princezná. Je vždy dobrá a milá...,pozitivna
2,Dobrá víla,Fiona je náladová. Toho super princa si nezasl...,negativna
3,Snehulienka,Je super.,pozitivna
4,Pinocchio,"Škoda reči, aj tak tento príspevok cenzúra vym...",negativna
5,Mabel,To čo má za muža. Takúto hroznú ozrutu doniesl...,negativna
6,Kyklop,Outfity čo nosí sú otrasné.,negativna
7,Merlin,Naša dobrá princezná je zároveň aj pekná.,pozitivna
8,Artuš,"Je dobrá, môže byť.",pozitivna
9,Jack,Naposledy mala otrasný účes. Tá farba vlasov j...,negativna


In [33]:
recenzie['Lema'] = ''

In [34]:
recenzie

,Hodnotitel,Text,Sentiment,Lema
0,Doris,Akože outfity má otrasné. Makeup je ale super ...,pozitivna,
1,Popoluška,Fiona je super princezná. Je vždy dobrá a milá...,pozitivna,
2,Dobrá víla,Fiona je náladová. Toho super princa si nezasl...,negativna,
3,Snehulienka,Je super.,pozitivna,
4,Pinocchio,"Škoda reči, aj tak tento príspevok cenzúra vym...",negativna,
5,Mabel,To čo má za muža. Takúto hroznú ozrutu doniesl...,negativna,
6,Kyklop,Outfity čo nosí sú otrasné.,negativna,
7,Merlin,Naša dobrá princezná je zároveň aj pekná.,pozitivna,
8,Artuš,"Je dobrá, môže byť.",pozitivna,
9,Jack,Naposledy mala otrasný účes. Tá farba vlasov j...,negativna,


In [35]:
for index, row in recenzie.iterrows():
  lemmy = daj_lemy(row['Text'])
  row['Lema'] = lemmy

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(recenzie['Lema'], recenzie['Sentiment'], test_size=0.4)

In [38]:
X_train

8                               byť dobrý , môcť byť .
1     fion byť super princezná . byť vždy dobrý a m...
7           náš dobrý princezná byť zároveň aj pekný .
2     fion byť náladový . to super princ si zaslúžiť .
6                       outfita čo nosiť byť otrasný .
3                                          byť super .
Name: Lema, dtype: object

In [39]:
X_test

9     naposledy mať otrasný účes . tá farba vlas by...
4     škoda reč , aj tak tento príspevok cenzúr vym...
5     to čo mať za muž . takýto hrozný ozrut donies...
0     akýže outfita mať otrasný . makeup byť ale su...
Name: Lema, dtype: object

In [40]:
y_train

8    pozitivna
1    pozitivna
7    pozitivna
2    negativna
6    negativna
3    pozitivna
Name: Sentiment, dtype: object

In [41]:
tfidf_vect = TfidfVectorizer()
X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [42]:
from sklearn.naive_bayes import MultinomialNB

In [43]:
clasificator = MultinomialNB().fit(X_train_tfidf,y_train)

In [44]:
y_priprava = tfidf_vect.transform(X_test)

In [45]:
y_pred = clasificator.predict(y_priprava)

In [46]:
from sklearn import metrics
print("Presnost", metrics.accuracy_score(y_test,y_pred))

Presnost 0.25
